We had to copy this code to fix a few bugs, plus I want to make some other modifications: When the arrow was fired it would stay in a fixed location and so the loop, which concludes when the arrow goes out of bounds gets stuck in an infinite loop. Another is that pits were being generated at the agents starting point, hardly fair on the guy (or gal, or...)! 

In [97]:
from agents2 import *

class WumpusEnvironment(XYEnvironment):
    pit_probability = 0.2  # Probability to spawn a pit in a location. (From Chapter 7.2)

    # Room should be 4x4 grid of rooms. The extra 2 for walls

    def __init__(self, width=6, height=6):
        super().__init__(width, height)
        self.init_world()

    def init_world(self):
        """Spawn items in the world based on probabilities from the book"""

        "WALLS"
        self.add_walls()

        "PITS"
        for x in range(self.x_start, self.x_end):
            for y in range(self.y_start, self.y_end):
                if random.random() < self.pit_probability:
                    if x == 1 and y == 1:
                        continue
                    self.add_thing(Pit(), (x, y), True)
                    self.add_thing(Breeze(), (x - 1, y), True)
                    self.add_thing(Breeze(), (x, y - 1), True)
                    self.add_thing(Breeze(), (x + 1, y), True)
                    self.add_thing(Breeze(), (x, y + 1), True)

        "WUMPUS"
        '''
        w_x, w_y = self.random_location_inbounds(exclude=(1, 1))
        self.add_thing(Wumpus(), (w_x, w_y), True)
        self.add_thing(Stench(), (w_x - 1, w_y), True)
        self.add_thing(Stench(), (w_x + 1, w_y), True)
        self.add_thing(Stench(), (w_x, w_y - 1), True)
        self.add_thing(Stench(), (w_x, w_y + 1), True)
        '''
        "GOLD"
        self.add_thing(Gold(), self.random_location_inbounds(exclude=(1, 1)), True)

    def get_world(self, show_walls=True):
        """Return the items in the world"""
        result = []
        x_start, y_start = (0, 0) if show_walls else (1, 1)

        if show_walls:
            x_end, y_end = self.width, self.height
        else:
            x_end, y_end = self.width - 1, self.height - 1

        for x in range(x_start, x_end):
            row = []
            for y in range(y_start, y_end):
                row.append(self.list_things_at((x, y)))
            result.append(row)
        return result

    def percepts_from(self, agent, location, tclass=Thing):
        """Return percepts from a given location,
        and replaces some items with percepts from chapter 7."""
        thing_percepts = {
            Gold: Glitter(),
            Wall: Bump(),
            Wumpus: Stench(),
            Pit: Breeze()}

        """Agents don't need to get their percepts"""
        thing_percepts[agent.__class__] = None

        """Gold only glitters in its cell"""
        if location != agent.location:
            thing_percepts[Gold] = None

        result = [thing_percepts.get(thing.__class__, thing) for thing in self.things
                  if thing.location == location and isinstance(thing, tclass)]
        return result if len(result) else [None]

    def percept(self, agent):
        """Return things in adjacent (not diagonal) cells of the agent.
        Result format: [Left, Right, Up, Down, Center / Current location]"""
        x, y = agent.location
        result = []
        result.append(self.percepts_from(agent, (x - 1, y)))
        result.append(self.percepts_from(agent, (x + 1, y)))
        result.append(self.percepts_from(agent, (x, y - 1)))
        result.append(self.percepts_from(agent, (x, y + 1)))
        result.append(self.percepts_from(agent, (x, y)))

        """The wumpus gives out a loud scream once it's killed."""
        wumpus = [thing for thing in self.things if isinstance(thing, Wumpus)]
        if len(wumpus) and not wumpus[0].alive and not wumpus[0].screamed:
            result[-1].append(Scream())
            wumpus[0].screamed = True

        return result

    def execute_action(self, agent, action):
        """Modify the state of the environment based on the agent's actions.
        Performance score taken directly out of the book."""
        if isinstance(agent, Explorer) and self.in_danger(agent):
            return
            
        agent.bump = False
        if action in ['TurnRight', 'TurnLeft', 'Forward', 'Grab']:
            super().execute_action(agent, action)
            agent.performance -= 1
        elif action == 'Climb':
            if agent.location == (1, 1):  # Agent can only climb out of (1,1)
                agent.performance += 1000 if Gold() in agent.holding else 0
                self.delete_thing(agent)
        elif action == 'Shoot':
            """The arrow travels straight down the path the agent is facing"""
            if agent.has_arrow:
                arrow_travel = agent.direction.move_forward(agent.location)
                while self.is_inbounds(arrow_travel):
                    wumpus = [thing for thing in self.list_things_at(arrow_travel)
                              if isinstance(thing, Wumpus)]
                    if len(wumpus):
                        wumpus[0].alive = False
                        break
                    arrow_travel = agent.direction.move_forward(arrow_travel)
                agent.has_arrow = False

    def in_danger(self, agent):
        """Check if Explorer is in danger (Pit or Wumpus), if he is, kill him"""
        for thing in self.list_things_at(agent.location):
            if isinstance(thing, Pit) or (isinstance(thing, Wumpus) and thing.alive):
                print("final location: ", agent.location)
                agent.alive = False
                agent.performance -= 1000
                agent.killed_by = thing.__class__.__name__
                return True
        return False

    def is_done(self):
        """The game is over when the Explorer is killed
        or if he climbs out of the cave only at (1,1)."""
        explorer = [agent for agent in self.agents if isinstance(agent, Explorer)]
        if len(explorer):
            if explorer[0].alive:
                return False
            else:
                print("Death by {} [-1000].".format(explorer[0].killed_by))
        else:
            print("Explorer climbed out {}."
                  .format("with Gold [+1000]!" if Gold() not in self.things else "without Gold [+0]"))
        return True
        

class Explorer(Agent):
    def __init__(self):
        super().__init__()
        self.holding = []
        self.has_arrow = True
        self.killed_by = ""
        self.direction = Direction("right")
        self.breeze_matrix = []
        self.create_matrices()

    def create_matrices(self):
        self.breeze_matrix = [[None] * 6 for _ in range(6)]

    def can_grab(self, thing):
        """Explorer can only grab gold"""
        return thing.__class__ == Gold

    def percept_location(self, location, i):
        if(i == 0):
            return [location[0] - 1, location[1]]
        if(i == 1):
            return [location[0] + 1, location[1]]
        if(i == 2):
            return [location[0], location[1] - 1]
        if(i == 3):
            return [location[0], location[1] + 1]
        if(i == 4):
            return [location[0], location[1]]
            
    def update_breeze_matrix(self, percept):
        print(self.location, percept)
        
        for i in range(len(percept)):
            p_loc = self.percept_location(self.location, i)
            found  = False
            for p in percept[i]:
                if(isinstance(p, Breeze)):
                    self.breeze_matrix[p_loc[0]][p_loc[1]] = True
                    found = True
            if not found:
                self.breeze_matrix[p_loc[0]][p_loc[1]] = False
                
        for r in self.breeze_matrix:
            print(r)

    # Converts percepts to actions. We NEED access to the class
    def program(self, percept):
        self.update_breeze_matrix(percept)
        action =  random.choice(['TurnRight', 'TurnLeft', 'Forward'])
        print(" I CHOOOOSE: ", action)
        return action


In [98]:
env = WumpusEnvironment()
agent = Explorer()
env.add_thing(agent, (1, 1), True)

print("-------------------------------")
world = env.get_world()
for w in world:
    print(w)
print("-------------------------------")

env.run()

-------------------------------
[[<Wall>], [<Wall>], [<Wall>], [<Wall>], [<Wall>], [<Wall>]]
[[<Wall>], [<Breeze>, <Explorer>], [<Pit>], [<Breeze>], [], [<Wall>]]
[[<Wall>], [], [<Breeze>], [], [<Breeze>, <Gold>], [<Wall>]]
[[<Wall>], [], [], [<Breeze>], [<Pit>], [<Wall>, <Breeze>]]
[[<Wall>], [], [], [], [<Breeze>], [<Wall>]]
[[<Wall>], [<Wall>], [<Wall>], [<Wall>], [<Wall>], [<Wall>]]
-------------------------------
(1, 1) [[<Bump>], [None], [<Bump>], [<Breeze>], [<Breeze>, None]]
[None, False, None, None, None, None]
[False, True, True, None, None, None]
[None, False, None, None, None, None]
[None, None, None, None, None, None]
[None, None, None, None, None, None]
[None, None, None, None, None, None]
 I CHOOOOSE:  TurnRight
(1, 1) [[<Bump>], [None], [<Bump>], [<Breeze>], [<Breeze>, None]]
[None, False, None, None, None, None]
[False, True, True, None, None, None]
[None, False, None, None, None, None]
[None, None, None, None, None, None]
[None, None, None, None, None, None]
[None, No